In [1]:
from pathlib import Path
from typing import Sequence

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
PATH_TRAIN = Path(r"data/synthetic_weather_train_4_4_0.9.csv")
PATH_TEST = Path(r"data/synthetic_weather_test_4_4_0.9.csv")
DAY_INDEX_TRAIN = [0, 1, 2, 3]
DAY_INDEX_TEST = [0, 1, 2, 3]

LEGEND = {
    'Irradiance 0': 'Módulo 1',
    'Irradiance 1': 'Módulo 2',
    'Irradiance 2': 'Módulo 3',
    'Irradiance 3': 'Módulo 4',
}

CB91_Blue = "#2CBDFE"
CB91_Green = "#47DBCD"
CB91_Pink = "#F3A0F2"
CB91_Purple = "#9D2EC5"
CB91_Violet = "#661D98"
CB91_Amber = "#F5B14C"

color_list = [
    CB91_Blue,
    CB91_Amber,
    CB91_Green,
    CB91_Violet,
    CB91_Pink,
    CB91_Purple,
]

plt.rcParams["axes.prop_cycle"] = plt.cycler(color=color_list)

plt.rc("text", usetex=True)
plt.rc("font", family="serif")

locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)

In [3]:
def read_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", drop=True, inplace=True)

    return df


def plot(df: pd.DataFrame):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    for col in df.columns:
        if "Amb" in col:
            continue  # Skip ambient temperatures
        ax.plot(df.index, df[col], label=LEGEND[col])
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)
        ax.set_ylabel("Irradiancia solar $\mathrm{(W/m^2)}$")
        ax.legend(loc="upper left")

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    

    return fig

def add_year_offset(df: pd.DataFrame) -> pd.DataFrame:
    df.index = df.index + pd.DateOffset(years=1)
    return df


def split_df_by_day(df: pd.DataFrame) -> Sequence[pd.DataFrame]:
    return [group[1] for group in df.groupby(df.index.date)]

In [4]:
df_train = read_csv(PATH_TRAIN)
df_test = read_csv(PATH_TEST)
df_test = add_year_offset(df_test)
df_train_day = split_df_by_day(df_train)
df_test_day = split_df_by_day(df_test)

In [5]:
for i, day in enumerate(DAY_INDEX_TRAIN, start=1):
    fig = plot(df_train_day[day])
    fig.savefig(f'output/fig_10_train_day_{i:02}.pdf', bbox_inches='tight')
    plt.close(fig)

In [6]:
for i, day in enumerate(DAY_INDEX_TEST, start=1):
    fig = plot(df_test_day[day])
    fig.savefig(f'output/fig_11_test_day_{i:02}.pdf', bbox_inches='tight')
    plt.close(fig)